# Converting PDF to Txt
This code is to convert all judgement PDFs into txt format

In [ ]:
import os
import shutil
import glob
from natsort import natsorted

from tqdm import tqdm
import pandas as pd

from pdf2image import convert_from_path
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter

In [ ]:
# Make a list of all paths of all PDFs in the directory
judgement_paths_assam = []
path ="Assam"
for root, dirs, files in os.walk(path):
    for file in files:
        if(file.endswith(".pdf")):
            judgement_paths_assam.append(os.path.join(root,file))
            
            
# Selecting only those PDFs that are judgements
judgements = []
for judgement in judgement_paths_assam:
    if ('judgment' in judgement.lower()):
        judgements.append(judgement)

In [ ]:
len(judgements)

In [ ]:
## Convert Judgment PDFs into txts
converted_cases = []
for judgement in tqdm(judgements):
    case_id = judgement.split(r'/')[-1].split('_')[0]
    if case_id in converted_cases:
        continue
        
    folder_path = judgement.split(r'/')
    folder_path.pop()
    folder_path = r"/".join(folder_path)+"/"

    pages = convert_from_path(judgement, 350, output_folder='/home/krishna/NLP_Justice/POCSO/tmp')
    
    i = 1
    for page in pages:
        image_name = "Page_" + str(i) + ".jpg"  
        page.save(image_name, "JPEG")
        i = i+1
    shutil.rmtree('tmp')
    os.makedirs('tmp')
        
    pics = glob.glob("*.jpg")
    pics = natsorted(pics)
    
    judgement_text = ''
    for pic in pics:
        text = pytesseract.image_to_string(Image.open(pic))
        os.remove(pic)
        judgement_text = judgement_text + text
    
    with open(folder_path+case_id+'.txt', 'w') as f:
        f.write(judgement_text)
        
    converted_cases.append(case_id)